In [42]:
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format
from constantes import WINSIG_SERVICIO_FARMACIA_CC_SIGCOM

In [43]:
def desglosar_centro_de_costo(desglose, total_dinero):
    con_dinero = desglose.copy()
    con_dinero['TOTAL_X_PORCENTAJE'] = con_dinero['PORCENTAJES'] * total_dinero

    return con_dinero


In [44]:
df_winsig = pd.read_excel('input\\Informe Winsig Septiembre 2022.xlsx', header=7)
df_winsig = df_winsig.dropna(axis=1, how='all').dropna(axis=0, how='all')
archivo_produccion = pd.ExcelFile('input\\output_producciones.xlsx')


In [55]:
desglose_pabellon = pd.read_excel(archivo_produccion, sheet_name='PABELLÓN')
total_pabellon = df_winsig.query('SERVICIO == "B. PABELLON"')['Gasto Servicio'].iloc[0]
con_gastos_pabellon = desglosar_centro_de_costo(desglose_pabellon, total_pabellon)

In [56]:
desglose_policlinico = pd.read_excel(archivo_produccion, sheet_name='CONSULTAS SIN MANEJO DEL DOLOR')
total_policlinico = df_winsig.query('SERVICIO == "POLICLÍNICO"')['Gasto Servicio'].iloc[0]
con_gastos_policlinico = desglosar_centro_de_costo(desglose_policlinico, total_policlinico)


In [65]:
con_gastos_pabellon_para_concatenar = con_gastos_pabellon.iloc[:, [0, 1, 2, -1, 3]]
con_gastos_pabellon_para_concatenar.iloc[:, [1, 2, -1]] = None
con_gastos_pabellon_para_concatenar.columns = df_winsig.columns

In [68]:
con_gastos_policlinico_para_concatenar = con_gastos_policlinico.iloc[:, [0, 1, 2, -1, 3]]
con_gastos_policlinico_para_concatenar.iloc[:, [1, 2, -1]] = None
con_gastos_policlinico_para_concatenar.columns = df_winsig.columns

In [80]:
winsig_concatenado = df_winsig.copy()
winsig_mas_desglose = pd.concat([winsig_concatenado, con_gastos_pabellon_para_concatenar])
winsig_mas_desglose = pd.concat([winsig_mas_desglose, con_gastos_policlinico_para_concatenar])
servicios_a_sacar = ['B. PABELLON', 'PABELLÓN', 'POLICLÍNICO', 'CONSULTAS SIN MANEJO DEL DOLOR']
servicios_a_dejar = ~(winsig_mas_desglose['SERVICIO'].isin(servicios_a_sacar))

winsig_final = winsig_mas_desglose[servicios_a_dejar]

In [81]:
index = winsig_final.set_index('SERVICIO')

In [83]:
index.to_json('prueba.json', indent=1, orient='index')

In [86]:
dict(index.index)

ValueError: dictionary update sequence element #0 has length 21; 2 is required

In [87]:
diciconario = {key: None for key in index.index}

In [ ]:
diciconario